# Fine tune CDLA pretrained model for KR court decisions

In [1]:
import subprocess, os, time
import fitz
import paddle
paddle.utils.run_check()

!export CUDA_VISIBLE_DEVICES=0

print(os.getcwd())

Running verify PaddlePaddle program ... 


I0429 01:26:40.927598 1927228 program_interpreter.cc:212] New Executor is Running.
W0429 01:26:40.928038 1927228 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0429 01:26:40.929040 1927228 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.


PaddlePaddle works well on 1 GPU.
PaddlePaddle is installed successfully! Let's start deep learning with PaddlePaddle now.
/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)


I0429 01:26:41.183156 1927228 interpreter_util.cc:624] Standalone Executor is Used.


/home/lstm/github/PaddleDetection/tuning


## Train

In [3]:
# Single GPU
command = (
    "python3 ../tools/train.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml",
    "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    "--eval",
    f"--use_vdl=True --vdl_log_dir=vdl_log/{time.strftime('%Y%m%d-%H%M%S')}",
    )
command = " ".join(command)
print(command)

python3 ../tools/train.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml --slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml --eval --use_vdl=True --vdl_log_dir=vdl_log/20240428-111721


In [4]:
!{command}
# output = subprocess.run(command, shell=True, capture_output=True)

/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
W0428 11:17:48.364167 1087718 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.5, Driver API Version: 12.4, Runtime API Version: 11.8
W0428 11:17:48.365097 1087718 gpu_resources.cc:164] device: 0, cuDNN Version: 8.8.
[04/28 11:17:49] ppdet.utils.checkpoint INFO: The shape [42] in pretrained weight head.head_cls0.bias is unmatched with the shape [43] in model head.head_cls0.bias. And the weight head.head_cls0.bias will not be loaded
[04/28 11:17:49] ppdet.utils.checkpoint INFO: The shape [42, 128, 1, 1] in pretrained weight head.head_cls0.weight is unmatched with the shape [43, 128, 1, 1] in model head.head_cls0.weight. And the weight head.head_cls0.weight will not be loaded
[04/28 11:17:49] ppdet.utils.checkpoint INFO: The shape [42] in pretrained weight head.head_cls1.bias is unmatched with the shape [43] in model head.head_cls1.bias. And the w

In [ ]:
# run visualdl in shell (from ./tuning dir)
# visualdl --logdir ./vdl_log/20240428-111721 --port 7070

## Tune

In [ ]:
# -r output/faster_rcnn_r50_1x_coco/10000

## Eval

In [ ]:

command = (
    "python3 ../tools/eval.py",
    "-c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml",
    "--slim_config ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml",
    # "-o weights=../output/best_model",
    "-o weights=../output/0.pdparams",
    # "use_gpu=true",
    # "--use_vdl=True --vdl_log_dir=vdl_log"
    )
command = " ".join(command)
print(command)

In [ ]:
!{command}

## Test 
##### Strategy: use command line method to test trained weights; create inference model for production.

#### Test prep (convert PDFs to images)

In [ ]:
# convert pdf to image and save

import fitz
import os


def convert_pdf_to_images(pdf_folder, output_folder):

    for pdf_path in os.listdir(pdf_folder):

        if not pdf_path.endswith('.pdf'):
            continue

        pdf_name = os.path.splitext(pdf_path)[0]
        # output_folder = os.path.join(pdf_folder, pdf_name)
        if not os.path.isdir(output_folder):
            os.mkdir(output_folder)

        pdf_path = os.path.join(pdf_folder, pdf_path)
        doc = fitz.open(pdf_path)

        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            img_name = os.path.join(output_folder, f'{page_num}.png')
            if not os.path.isfile(img_name):
                page.get_pixmap().save(img_name)
            else:
                print(f"{img_name} exists!")

        doc.close()

In [ ]:
pdf_folder = './testdata/'
output_folder = './testdata/'
# pdf_folder = './tuning/testdata/'
convert_pdf_to_images(pdf_folder, output_folder)

#### Test inference

In [ ]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"
# opts = "weights=./trained_weights/output_20240423/best_model/model.pdparams"

inference_datafolder = './testdata'
testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'

inference_outputfolder = './testoutput'
testout_img_folder = 'adobe_대법원_2020도1007_판결서'

In [ ]:
args = {
    'config': configfile,
    'slim_config': slim_configfile,
    'infer_dir': f'{inference_datafolder}/{testdoc_img_folder}',
    'infer_img': None,
    'output_dir': f'{inference_datafolder}/{testout_img_folder}',
    'draw_threshold': 0.5, # default
    'use_vdl': True,
    'vdl_log_dir': '/vdl_log_test',
    'save_results': False,
    # slice_infer, action='store_true',
    # slice_size, nargs='+,
    # overlap_ratio, nargs='+',
    # combine_method: None,
    # match_threshold: None,
    # match_metric: None,
    'visualize': True
}

print(json.dumps(args, indent=4))

In [ ]:
# convert image to bytes
datapath = 'testdata'
filelist = os.listdir(datapath)

def pdf2byte(filename):
    pdf = fitz.open(os.path.join(datapath, filename))
    for pg in range(pdf.pageCount):
        # convert pdf to image and store in local variable
        page = pdf[pg]
        pix = page.getPixmap()
        img_data = page.getPixmap().tobytes("png")
        # stor pix in local variable


In [32]:
configfile = "../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml"
slim_configfile = " ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml"

inference_datafolder = './testdata'
testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'
weight2use = '20240428'
output_dir = './testresults'

command = (
    "python3 ../tools/infer.py",
    f"-c {configfile}",
    f"--slim_config {slim_configfile}",
    f"--infer_dir={inference_datafolder}/{testdoc_img_folder}",
    # f"--infer_img={inference_datafolder}/{testdoc_img_folder}/0.png",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    "--draw_threshold=0.5",
    # "use_gpu=true",
    "--use_vdl=True",
    "--vdl_log_dir=./vdl_log_test",
    f"--output_dir={output_dir}"
    )
command = " ".join(command)
print(command)


python3 ../tools/infer.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml --slim_config  ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml --infer_dir=./testdata/adobe_대법원_2020도1007_판결서 -o weights=./trained_weights/output_20240428/best_model/model.pdparams --draw_threshold=0.5 --use_vdl=True --vdl_log_dir=./vdl_log_test --output_dir=./testresults


In [ ]:

!{command}


## Deploy model (use PaddleOCR)

#### Model export (distilled model)
https://github.com/PaddlePaddle/PaddleOCR/blob/release/2.7/ppstructure/docs/quickstart_en.md

In [40]:
#TODO

model_outpath = './exported_model'

command = (
    "python3 ../tools/export_model.py",
    f"-c {configfile}",
    f"--slim_config {slim_configfile}",
    f"-o weights=./trained_weights/output_{weight2use}/best_model/model.pdparams",
    f"--output_dir={model_outpath}"
    # f"-o export.benchmark=True"
    )
command = " ".join(command)
print(command)

python3 ../tools/export_model.py -c ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_train.yml --slim_config  ../configs/picodet/legacy_model/application/layout_analysis/krcase_layout_distill.yml -o weights=./trained_weights/output_20240428/best_model/model.pdparams --output_dir=./exported_model


In [41]:
!{command}

5063.00s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/bin/bash: /home/lstm/miniconda3/envs/pp261/lib/libtinfo.so.6: no version information available (required by /bin/bash)
[04/29 02:51:01] ppdet.utils.checkpoint INFO: Finish loading model weights: ./trained_weights/output_20240428/best_model/model.pdparams
loading annotations into memory...
Done (t=0.20s)
creating index...
index created!
[04/29 02:51:01] ppdet.engine INFO: Export inference config file to ./exported_model/krcase_layout_train/infer_cfg.yml
I0429 02:51:04.794533 2030709 program_interpreter.cc:212] New Executor is Running.
[04/29 02:51:04] ppdet.engine INFO: Export model and saved in ./exported_model/krcase_layout_train


#### Model inference (distilled model)

| Field | Description | Default |
|-|-|-|  
| output | result save path | ./output/table |
| table_max_len | long side of the image resize in table structure model | 488 |
| table_model_dir | Table structure model inference model path | None |
| table_char_dict_path | The dictionary path of table structure model | ../ppocr/utils/dict/table_structure_dict.txt |
| merge_no_span_structure | In the table recognition model, whether to merge '<td>' and '</td>' | False |
| layout_model_dir | Layout analysis model inference model path | None |
| layout_dict_path | The dictionary path of layout analysis model | ../ppocr/utils/dict/layout_publaynet_dict.txt |
| layout_score_threshold | The box threshold path of layout analysis model | 0.5 |
| layout_nms_threshold | The nms threshold path of layout analysis model | 0.5 |
| kie_algorithm | kie model algorithm | LayoutXLM |
| ser_model_dir | Ser model inference model path | None |
| ser_dict_path | The dictionary path of Ser model | ../train_data/XFUND/class_list_xfun.txt |
| mode | structure or kie | structure |
| image_orientation | Whether to perform image orientation classification in forward | False |
| layout | Whether to perform layout analysis in forward | True |
| table | Whether to perform table recognition in forward | True |
| ocr | Whether to perform ocr for non-table areas in layout analysis. When layout is False, it will be automatically set to False | True |
| recovery | Whether to perform layout recovery in forward | False |
| save_pdf | Whether to convert docx to pdf when recovery | False |
| structure_version | Structure version, optional PP-structure and PP-structurev2 | PP-structure |


## Inference with deployed model (Python)

#### using PPStructure class

In [38]:
import os
import cv2
import matplotlib.pyplot as plt
from paddleocr import PPStructure,draw_structure_result,save_structure_res

from PIL import Image
from IPython.display import display


inference_datafolder = './testdata'
testdoc_img_folder = 'adobe_대법원_2020도1007_판결서'

inference_outputfolder = './testresults_deployment'
testout_img_folder = 'adobe_대법원_2020도1007_판결서'

font_path = '../NotoSansKR-Regular.otf' # PaddleOCR

imgs = os.listdir(f"{inference_datafolder}/{testdoc_img_folder}")
imgs


['3.png', '2.png', '1.png', '0.png']

In [39]:
# get layout
# layout_engine = PPStructure(table=False, ocr=False, show_log=True) #, image_orientation=True)

layout_engine = PPStructure(
    output=inference_outputfolder,
    table_max_len=None, # 488
    table_model_dir=None,
    table_char_dict_path=None, # ../ppocr/utils/dict/table_structure_dict.txt
    merge_no_span_structure=False,
    layout_model_dir="./exported_model/krcase_layout_train", #None, # use <model_outpath> to use exported model
    layout_dict_path=None, # use <layout_dict_path> ? (dict should be in exported model)
    layout_score_threshold=0.4, # 0.5
    layout_nms_threshold= 0.3, # 0.5
    kie_algorithm=None, # LayoutXLM
    ser_model_dir=None,
    ser_dict_path=None, #../train_data/XFUND/class_list_xfun.txt
    mode="structure",
    image_orientation=False,
    layout=True,
    table=False,
    ocr=False,
    recovery=False,
    save_pdf=False,
    structure_version="PP-StructureV2",
    show_log=True
    )

for imgpath in imgs:

    img = cv2.imread(f"{inference_datafolder}/{testdoc_img_folder}/{imgpath}")
    # display(img)
    result = layout_engine(img)
    save_structure_res(result, inference_outputfolder, os.path.basename(imgpath).split('.')[0])

    im_show = draw_structure_result(img, result,font_path=font_path)
    imgbyte = Image.fromarray(im_show)

    height, width = im_show.shape[:2]
    mid_x = width // 2

    # Crop left side
    box = (0, 0, mid_x, height)
    img_left = imgbyte.crop(box)
    if not os.path.isdir(f"{inference_outputfolder}/{testout_img_folder}"):
        os.mkdir(f"{inference_outputfolder}/{testout_img_folder}")


    img_left.save(f"{inference_outputfolder}/{testout_img_folder}/{imgpath}")

    # display image
    display(img_left)

    # diplay image with matplotlib without axis (from im_show ndarray)
    # dpi = 200
    # height, width, depth = im_show.shape
    # figsize = width / float(dpi), height / float(dpi)
    # fig = plt.figure(figsize=figsize, dpi=dpi)
    # ax = fig.add_axes([0, 0, 1, 1])
    # ax.axis('off') # hide axis
    # plt.imshow(im_show)

[2024/04/29 02:50:32] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir=None, cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/home/lstm/.paddleocr/whl/det/ch/ch_PP-OCRv4_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set='to

ValueError: (InvalidArgument) The input of Op(Conv) should be a 4-D or 5-D Tensor. But received: input's dimension is -1, input's shape is [].
  [Hint: Expected in_dims.size() == 4 || in_dims.size() == 5 == true, but received in_dims.size() == 4 || in_dims.size() == 5:0 != true:1.] (at /paddle/paddle/phi/kernels/cpu/conv_util.h:137)
  [operator < fused_conv2d_add_act > error]